<a href="https://colab.research.google.com/github/kyle-gao/ML_ipynb/blob/master/Pytorch_CNN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import torch
import torch.nn as nn
import torchvision
from torchsummary import summary

In [96]:
transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (1), (0.5))
                             ])

batch_size = 32

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('/files/', train=True, download=True,
                             transform = transform), batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.FashionMNIST('/files/', train=False, download=True, 
                                    transform = transform),  batch_size=batch_size, shuffle=True)

In [97]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()

    self.conv1 = nn.Conv2d(1,32,3,1) #in Channel, out Channel, kernel, stride
    self.conv2 = nn.Conv2d(32,64,3,1)

    self.dropout1 = nn.Dropout2d(0.2)
    self.dropout2 = nn.Dropout(0.2)

    self.dense1 = nn.Linear(1600, 128)
    self.dense2 = nn.Linear(128,10)
  
  def forward(self,x):
    x = self.conv1(x)
    x = nn.functional.relu(x)
    x = x = nn.functional.max_pool2d(x,2)
    x = self.dropout1(x)

    x = self.conv2(x)
    x = nn.functional.relu(x)
    x = nn.functional.max_pool2d(x,2)
    x = torch.flatten(x,1)
    #print(x.shape) #print shape to caculate dense input dim
    x = self.dense1(x)
    x = self.dropout2(x)
    x = self.dense2(x)
    return nn.functional.log_softmax(x, dim=1)

conv = Model()
print(conv)
from torchsummary import summary

summary(conv, (1, 28, 28))

Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=1600, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=10, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
         Dropout2d-2           [-1, 32, 13, 13]               0
            Conv2d-3           [-1, 64, 11, 11]          18,496
            Linear-4                  [-1, 128]         204,928
           Dropout-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 225,034
Trainable params: 225,034
Non-trainable params: 0
----------------------------------------------------------------


In [98]:
it = iter(train_loader)
test_batch, _ = next(it)
print(conv.forward(test_batch).shape)

torch.Size([32, 10])


In [109]:
def train(model, train_loader):

  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(conv.parameters())

  for epoch in range(epochs):
    epoch_loss = 0.0
    i = 0
    for _,data in enumerate(train_loader, 0):
      i = i + 1
      inputs, labels = data

      optimizer.zero_grad()

      outputs = model(inputs)

      loss = loss_fn(outputs,labels)
      loss.backward()
      optimizer.step()

      epoch_loss = epoch_loss + loss


    print('epoch: %d loss: %.3f' %
    (epoch + 1, epoch_loss / i))

    

In [110]:
train(conv,train_loader)

epoch: 1 loss: 0.656
epoch: 2 loss: 0.396


In [111]:
def evaluate(model):
    correct = 0 
    model.eval()
    for img, labels in test_loader:
        #print(test_imgs.shape)
        output = model(img)
        predicted = torch.max(output,1)[1]
        correct += (predicted == labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*batch_size)))

In [112]:
evaluate(conv)

Test accuracy:0.862% 
